In [8]:
# --- 1. 라이브러리 설치 및 임포트 ---

# .env 파일을 사용하기 위한 라이브러리를 설치합니다.
!pip install python-dotenv

import pandas as pd
import requests
import time
from tqdm import tqdm
import os
from dotenv import load_dotenv # .env 파일을 읽기 위한 라이브러리

# tqdm 라이브러리와 pandas의 연동을 설정합니다.
tqdm.pandas()

In [9]:
# --- 2. .env 파일에서 API 키 로드 ---

# .env 파일에 정의된 환경 변수를 현재 세션으로 로드합니다.
load_dotenv()

# os.getenv() 함수를 사용하여 "KAKAO_REST_API_KEY" 라는 이름의 환경 변수 값을 가져옵니다.
KAKAO_REST_API_KEY = os.getenv("KAKAO_REST_API_KEY")

print("KAKAO_REST_API_KEY:", KAKAO_REST_API_KEY)

KAKAO_REST_API_KEY: f760c2b9b524c0fc455b78fd2a58b773


In [22]:
# --- 3. 설정 및 데이터 로드 ---

TRAIN_DATA_PATH = '../data/raw/train.csv'
OUTPUT_DIR = '../data/processed/'
OUTPUT_FILENAME = 'train_coordinate_geocoded.csv'
OUTPUT_PATH = os.path.join(OUTPUT_DIR, OUTPUT_FILENAME)

print(f"'{TRAIN_DATA_PATH}'에서 원본 데이터를 로드합니다...")

try:
    train_df = pd.read_csv(TRAIN_DATA_PATH)
    print("데이터 로드 완료.\n")
    initial_missing_count = train_df['좌표X'].isnull().sum()
    print(f"작업 전 '좌표X'의 결측치 개수: {initial_missing_count}")
except FileNotFoundError:
    print(f"오류: '{TRAIN_DATA_PATH}' 파일을 찾을 수 없습니다.")
    train_df = None

'../data/raw/train.csv'에서 원본 데이터를 로드합니다...


/tmp/ipykernel_87372/1257457201.py:11: DtypeWarning: Columns (16,17,36) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(TRAIN_DATA_PATH)


데이터 로드 완료.

작업 전 '좌표X'의 결측치 개수: 869670


In [23]:
# --- 4. 지오코딩 함수 정의 ---

def get_coordinates_kakao(address, api_key):
    """
    카카오 API를 활용하여 한글 주소(법정동+번지 등) → 좌표(경도, 위도) 변환
    """
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {"query": address}
    try:
        response = requests.get(url, headers=headers, params=params, timeout=3)
        response.raise_for_status()
        data = response.json()
        if data['documents']:
            lon = float(data['documents'][0]['x'])
            lat = float(data['documents'][0]['y'])
            return lon, lat
        else:
            return None, None
    except Exception as e:
        print(f"[API오류] 주소 변환 중 에러: {e} (address: {address})")
        return None, None

In [24]:
# --- 4-1. 테스트용 샘플 1건 선택 ---

# 좌표 결측치 중 첫 번째 행 선택
test_row = train_df[train_df['좌표X'].isnull()].iloc[0]

# --- 4-2. 주소 조합 (도로명+시군구+번지 우선) ---
def make_address(row):
    return f"{row['시군구']} {row['도로명']}"

address = make_address(test_row)
print("테스트 주소:", address)

# --- 4-3. 지오코딩 함수로 좌표 조회 ---
lon, lat = get_coordinates_kakao(address, KAKAO_REST_API_KEY)
print("조회 결과 좌표X(경도):", lon)
print("조회 결과 좌표Y(위도):", lat)

테스트 주소: 서울특별시 강남구 개포동 삼성로 14
조회 결과 좌표X(경도): 127.068451024326
조회 결과 좌표Y(위도): 37.4861548149371


In [ ]:
# --- 4.2 테스트용 상위 10개 결측치(좌표) 보간 실행 및 저장 ---

if train_df is not None and KAKAO_REST_API_KEY:
    # 결측치 행만 추출 (시간/비용 절감)
    train_df_to_geocode = train_df[train_df['좌표X'].isnull()].copy()
    
    # 10개 샘플만 우선 테스트 (결측치가 10개 미만이면 해당 개수만)
    test_sample_n = 10
    if not train_df_to_geocode.empty:
        train_df_to_geocode = train_df_to_geocode.iloc[:test_sample_n]
        print(f"\n총 {len(train_df_to_geocode)}개 주소(샘플)에 대해 지오코딩을 시작합니다...")

        # 도로명 결측치 없음 → 시군구 + 도로명 조합
        def make_address(row):
            return f"{row['시군구']} {row['도로명']}"
        
        train_df_to_geocode['full_address'] = train_df_to_geocode.apply(make_address, axis=1)

        # tqdm을 사용하여 진행 상황 표시
        def fetch_coord(row):
            time.sleep(0.05)  # 카카오 QPS 10회/초 제한
            return get_coordinates_kakao(row['full_address'], KAKAO_REST_API_KEY)

        # 각 행별로 좌표 조회(튜플 반환) → 바로 좌표X/Y에 저장
        coords = train_df_to_geocode.progress_apply(fetch_coord, axis=1)
        
        train_df_to_geocode['좌표X'] = coords.apply(lambda x: x[0])
        train_df_to_geocode['좌표Y'] = coords.apply(lambda x: x[1])

        # 원본 train_df에 해당 index만 값 덮어쓰기
        train_df.loc[train_df_to_geocode.index, ['좌표X', '좌표Y']] = train_df_to_geocode[['좌표X', '좌표Y']]

        print("지오코딩 및 결측치(샘플 10개) 업데이트 완료.")
    else:
        print("좌표 결측치가 존재하지 않습니다.")

    # 작업 후 남은 결측치 확인
    final_missing_count = train_df['좌표X'].isnull().sum()
    print(f"작업 후 '좌표X' 결측치 개수: {final_missing_count}")

    # 결과 저장
    #os.makedirs(OUTPUT_DIR, exist_ok=True)
    #train_df.to_csv(OUTPUT_PATH, index=False)
    print(f"처리 완료: '{OUTPUT_PATH}'에 저장됨.")
    
    # 좌표가 새로 추가된 행만 확인용으로 따로 저장
    sample_output_path = os.path.join(OUTPUT_DIR, 'train_geocoded_sample.csv')
    train_df_to_geocode.to_csv(sample_output_path, index=False)
    print(f"샘플 결과 저장 완료: '{sample_output_path}'")



elif train_df is None:
    print("[오류] 데이터가 로드되지 않았습니다. 'train_df'가 None입니다.")
    pass
else:
    print("[오류] .env 파일에 KAKAO_REST_API_KEY가 필요합니다.")



총 10개 주소(샘플)에 대해 지오코딩을 시작합니다...


100%|██████████| 10/10 [00:01<00:00,  7.17it/s]

지오코딩 및 결측치(샘플 10개) 업데이트 완료.
작업 후 '좌표X' 결측치 개수: 869650
처리 완료: '../data/processed/train_coordinate_geocoded.csv'에 저장됨.
샘플 결과 저장 완료: '../data/processed/train_geocoded_sample.csv'


In [ ]:
# --- 5. 결측치(좌표) 보간 실행 및 저장 ---

if train_df is not None and KAKAO_REST_API_KEY:
    # 좌표 결측치 행 전체 추출
    train_df_to_geocode = train_df[train_df['좌표X'].isnull()].copy()
    
    if not train_df_to_geocode.empty:
        print(f"\n총 {len(train_df_to_geocode)}개 주소에 대해 지오코딩을 시작합니다...")

        # 도로명 결측치 없음 → 시군구 + 도로명 조합
        def make_address(row):
            return f"{row['시군구']} {row['도로명']}"
        
        train_df_to_geocode['full_address'] = train_df_to_geocode.apply(make_address, axis=1)

        # tqdm을 사용하여 진행 상황 표시
        def fetch_coord(row):
            time.sleep(0.05)  # 카카오 QPS 10회/초 제한
            return get_coordinates_kakao(row['full_address'], KAKAO_REST_API_KEY)

        # 모든 결측치 행에 대해 좌표 채우기
        coords = train_df_to_geocode.progress_apply(fetch_coord, axis=1)
        
        train_df_to_geocode['좌표X'] = coords.apply(lambda x: x[0])
        train_df_to_geocode['좌표Y'] = coords.apply(lambda x: x[1])

        # 원본 train_df에 해당 index만 값 덮어쓰기
        train_df.loc[train_df_to_geocode.index, ['좌표X', '좌표Y']] = train_df_to_geocode[['좌표X', '좌표Y']]

        print("지오코딩 및 결측치(전체) 업데이트 완료.")
    else:
        print("좌표 결측치가 존재하지 않습니다.")

    # 작업 후 남은 결측치 확인
    final_missing_count = train_df['좌표X'].isnull().sum()
    print(f"작업 후 '좌표X' 결측치 개수: {final_missing_count}")

    # 결과 저장
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    train_df.to_csv(OUTPUT_PATH, index=False)
    print(f"처리 완료: '{OUTPUT_PATH}'에 저장됨.")
    
    # 좌표가 새로 추가된 행만 확인용으로 따로 저장
    sample_output_path = os.path.join(OUTPUT_DIR, 'train_geocoded_all.csv')
    train_df_to_geocode.to_csv(sample_output_path, index=False)
    print(f"전체 보간 결과 저장 완료: '{sample_output_path}'")

elif train_df is None:
    print("[오류] 데이터가 로드되지 않았습니다. 'train_df'가 None입니다.")
    pass
else:
    print("[오류] .env 파일에 KAKAO_REST_API_KEY가 필요합니다.")


총 869670개 주소에 대해 지오코딩을 시작합니다...


  0%|          | 197/869670 [00:23<29:23:25,  8.22it/s]


KeyboardInterrupt: 